In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf

from tensorflow.keras import Sequential

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPool3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [0]:
data_path = 'data'
labels = pd.read_csv(os.path.join(data_path, 'adni_demographic_master_kaggle.csv'))

In [0]:
def load_datasets(type: str):
    if type not in ['train', 'test', 'valid']: raise Exception('Unsupported dataset type')
    train_valid_test = 0 if type == 'train' else 1 if type == 'valid' else 2
    i = 1
    dataset = np.load(os.path.join(data_path, f'img_array_{type}_6k_{i}.npy'))
    while True:
        try:
            i += 1
            dataset = np.vstack((dataset, np.load(os.path.join(data_path, f'img_array_{type}_6k_{i}.npy'))))
        except FileNotFoundError:
            print(f'Loaded all {type} datasets')
            break
    # dataset = np.expand_dims(dataset, axis=1)
    dataset = np.reshape(dataset, (-1, 62, 96, 96, 1))
    for i in range(dataset.shape[0]):
        dataset[i] = dataset[i]/np.amax(dataset[i])
    return dataset, np.eye(3)[labels[labels.train_valid_test == train_valid_test].diagnosis - 1]

In [0]:
test_data, test_labels = load_datasets('test')
train_data, train_labels = load_datasets('train')
valid_data, valid_data = load_datasets('valid')

Loaded all valid datasets


In [0]:
def create_model():
    kernel_regularizer = tf.keras.regularizers.l2(0.001)
    he_normal = tf.keras.initializers.he_normal(seed=0)
    lecun_normal = tf.keras.initializers.lecun_normal(seed=0)

    input_3d = (62, 96, 96, 1)
    pool_3d = (2, 2, 2)

    model = Sequential()
    
    model.add(tf.keras.layers.InputLayer(input_shape=input_3d))
    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool1'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool2'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool3'))

    model.add(Flatten())
    model.add(Dense(1024, name='dense1', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout1'))

    model.add(Dense(512, activation='relu', name='dense2', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout2'))

    model.add(Dense(3, activation='softmax', name='softmax', kernel_initializer=lecun_normal))

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])

    return model

In [0]:
reducelr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                                         factor=0.2,
                                                         patience=5,
                                                         min_delta=0.01,
                                                         verbose=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join('weights', 'best_weights.h5'),
                                                         monitor='val_categorical_accuracy',
                                                         verbose=1,
                                                         save_best_only=True,
                                                         mode='max')

callbacks_list = [checkpoint_callback, reducelr_callback]

In [0]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 60, 94, 94, 8)     224       
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 94, 94, 8)     32        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 60, 94, 94, 8)     0         
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 47, 47, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 28, 45, 45, 8)     1736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 45, 45, 8)     32        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 45, 45, 8)     0

In [0]:
history = model.fit(x=train_data, y=train_labels, validation_data=(valid_data, valid_labels), epochs=25, callbacks=callbacks_list, verbose=1)

Epoch 1/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3448 - loss: 13.1440
Epoch 00001: val_categorical_accuracy improved from -inf to 0.21839, saving model to weights/best_weights.h5
11/11 [==============================] - 16s 1s/step - categorical_accuracy: 0.3448 - loss: 13.1440 - val_categorical_accuracy: 0.2184 - val_loss: 4.5574 - lr: 0.0010
Epoch 2/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3621 - loss: 7.0886
Epoch 00002: val_categorical_accuracy improved from 0.21839 to 0.37931, saving model to weights/best_weights.h5
11/11 [==============================] - 6s 522ms/step - categorical_accuracy: 0.3621 - loss: 7.0886 - val_categorical_accuracy: 0.3793 - val_loss: 5.1208 - lr: 0.0010
Epoch 3/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3477 - loss: 5.9051
Epoch 00003: val_categorical_accuracy did not improve from 0.37931
11/11 [==============================] - 5s 480ms/step - c


Epoch 00005: val_categorical_accuracy did not improve from 0.40230
11/11 [==============================] - 5s 498ms/step - categorical_accuracy: 0.3822 - loss: 4.7358 - val_categorical_accuracy: 0.3563 - val_loss: 4.1793 - lr: 0.0010
Epoch 6/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.4167 - loss: 4.4390WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.



Epoch 00006: val_categorical_accuracy improved from 0.40230 to 0.41379, saving model to weights/best_weights.h5
11/11 [==============================] - 6s 526ms/step - categorical_accuracy: 0.4167 - loss: 4.4390 - val_categorical_accuracy: 0.4138 - val_loss: 4.1360 - lr: 0.0010
Epoch 7/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3678 - loss: 4.2850WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.



Epoch 00007: val_categorical_accuracy did not improve from 0.41379
11/11 [==============================] - 5s 473ms/step - categorical_accuracy: 0.3678 - loss: 4.2850 - val_categorical_accuracy: 0.4023 - val_loss: 4.0965 - lr: 0.0010
Epoch 8/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.2989 - loss: 4.1821WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.



Epoch 00008: val_categorical_accuracy did not improve from 0.41379
11/11 [==============================] - 5s 475ms/step - categorical_accuracy: 0.2989 - loss: 4.1821 - val_categorical_accuracy: 0.2184 - val_loss: 4.0698 - lr: 0.0010
Epoch 9/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3333 - loss: 4.1123WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


...


Epoch 00021: val_categorical_accuracy did not improve from 0.41379

Epoch 00021: ReduceLROnPlateau reducing learning rate to 8.000000525498762e-06.
11/11 [==============================] - 6s 510ms/step - categorical_accuracy: 0.3621 - loss: 3.8645 - val_categorical_accuracy: 0.3563 - val_loss: 3.8780 - lr: 4.0000e-05
Epoch 22/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3678 - loss: 3.8652WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more 


Epoch 00022: val_categorical_accuracy did not improve from 0.41379
11/11 [==============================] - 5s 465ms/step - categorical_accuracy: 0.3678 - loss: 3.8652 - val_categorical_accuracy: 0.3563 - val_loss: 3.8775 - lr: 8.0000e-06
Epoch 23/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3764 - loss: 3.8617WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.



Epoch 00023: val_categorical_accuracy did not improve from 0.41379
11/11 [==============================] - 5s 474ms/step - categorical_accuracy: 0.3764 - loss: 3.8617 - val_categorical_accuracy: 0.3563 - val_loss: 3.8770 - lr: 8.0000e-06
Epoch 24/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3678 - loss: 3.8601WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.



Epoch 00024: val_categorical_accuracy did not improve from 0.41379
11/11 [==============================] - 5s 462ms/step - categorical_accuracy: 0.3678 - loss: 3.8601 - val_categorical_accuracy: 0.3563 - val_loss: 3.8765 - lr: 8.0000e-06
Epoch 25/25
11/11 [==============================] - ETA: 0s - categorical_accuracy: 0.3851 - loss: 3.8627WARNING:tensorflow:5 out of the last 13 calls to <function Model.make_test_function.<locals>.test_function at 0x7f90d3013d90> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.



Epoch 00025: val_categorical_accuracy did not improve from 0.41379
11/11 [==============================] - 5s 478ms/step - categorical_accuracy: 0.3851 - loss: 3.8627 - val_categorical_accuracy: 0.3563 - val_loss: 3.8757 - lr: 8.0000e-06
